#### CASCODE COMMON-SOURCE (11)
### Common source amplifier with cascode transistor or cascode current mirror

* The cascode transistor M2 for the common source amplifier stages M1 is an impedance converter. 
  * The input impedance of the cascode transitor M2 is low $\frac{1}{g_m}$ while the output impedance is large ${g_m}{r_{cascode}}{r_o}$
  * The M1 is sized for the required $g_m$, $r_o$ and the M2 is sized for overall desired $r_{cascode}$ 
  * The operating points are chosen as per the desired bias current in the branch which could be set by another active load device not shown here. The biasing of the cascode device could be generated by another bias voltage source, but for the purpose of the block , it is defined at a level of $2*V_{DSAT}+V_{THN}$ of about 1V. Hence, specifications: 
    * $V_{BIAS}$ = 1V
    * $I_{BIAS}$ = 1uA
    * $g_{m, M1}$ = 10mS
    * $r_{o, M1}$ = $\frac{1}{g_{ds, M1}}$ = 100KOhms
    * $r_{cascode, M2}$ = $\frac{1}{g_{ds, M2}}$ = 50KOhms

  * To verify, overall small signal output impedance is ${10{e}^{-3}}*{100e^3}*{5e^3}$ = $5e^7$ Ohms. 

In [2]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

In [3]:
nfet_01v8_lvt = lk('../techsweep/simulation/nfet_01v8_lvt.mat')

#### Specifications

In [4]:
gm = 10e-3
ro = 100e3
rcascode = 50e3

#### Design Choices

In [32]:
l = 0.15
# Large gmid for input device for higher gain in weak inversion
# & small gmid for cascode device to keep in deep saturation
gm_id_in = np.array([20, 10])
gm_id_casc = np.array([6, 15])

#### Biasing

In [55]:
vin_bias = 0.3
vcasc_bias = 1

#### Sizing and Benchmarking

In [54]:
# Sizing the input active device for amplications
id = gm/gm_id_in
# Look up JD for the requried GM_ID and L.
jd = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_id_in, L=l, vgs_w=vin_bias)
# Get W from the ID and JD
w = id/jd
# Get the gds of the input device from looking up gmid for the length and then multiplying it with W.
gds_in = nfet_01v8_lvt.lookup('GDS_W', GM_ID=gm_id_in, L=l, vgs_w=vin_bias)*w
## The output impedance is inverse of the conductance.
rout_in = 1/gds_in
VTH_nfet = nfet_01v8_lvt.lookup('VT', GM_ID=gm_id_in, L=l, vg_ws=vin_bias); print(f"VT: ",VTH_nfet)
VDSAT_in = 2/(gm_id_in)
VGS_in = nfet_01v8_lvt.lookup('VGS', GM_ID=gm_id_in, L=l, vgs_w=vin_bias); print(f"VGS: ",VGS_in)

# print(f" Checking the availability of the data from datafame: id", id, jd, w)
# Printing for one column
# df = pd.DataFrame([gm_id_in, id, jd],['gm_id', 'id', 'jd'], columns=['option1'])
df = pd.DataFrame([gm_id_in, id, jd, w, gds_in, rout_in, VDSAT_in],
                    ['gm_id', 'id', 'jd', 'w', 'gds', 'ROUT', "VDSAT"], 
                        columns=[f'Option1 : l = {l}', f'Option2: l = {l}'])
df

VT:  []
VGS:  []


,Option1 : l = 0.15,Option2: l = 0.15
gm_id,20.000000,10.000000
id,0.000500,0.001000
jd,0.000003,0.000032
w,162.475199,30.942530
gds,0.000565,0.000599
ROUT,1770.212256,1670.351713
VDSAT,0.100000,0.200000


### Sizing and Benchmarking sizes for cascode transistor

In [67]:
id_casc = id
print(id_casc)
gm_casc = nfet_01v8_lvt.lookup('GM_ID', ID=id_casc, L=l, vgs_w=vcasc_bias); print(gm_casc)
# jd_casc = nfet_01v8_lvt.lookup('ID_W', GM_ID=gm_casc/id_casc, L=1, vgs_w=vcasc_bias)


[0.0005 0.001 ]
[25.69245463 25.69723618 25.69068077 25.68249    25.66331199 25.65301205
 25.63451777 25.60535117 25.56104446 25.50875885 25.43375394 25.33444816
 25.21111111 25.03556609 24.82122994 24.54155955 24.22180801 23.83495146
 23.41118188 22.97147385 22.53028264 22.05549846 21.46690519 20.64260563
 19.49946752 18.04260985 16.34985928 14.56390646 12.81863769 11.21517028
  9.79154258  8.56534091  7.52808989  6.65692957  5.93065187  5.32063645
  4.80985766  4.37772305  4.00887069  3.69130637  3.41747022  3.17837555
  2.96806967  2.78140477  2.6149637   2.46495558  2.33116279  2.20913884
  2.09741882  1.9960443   1.90278824  1.81726619  1.7377839   1.66424802
  1.59531349  1.5316687   1.47214076  1.41572398  1.36393443  1.31501057
  1.2691914   1.22519841  1.18383061  1.14539504  1.10863636  1.07341884
  1.04005168  1.00881612  0.97870598  0.95001999  0.92268645  0.89690722
  0.87191934]


#### Spice Netlist

In [39]:
%%writefile ./sizing_cascoded_common_source.spice
** Cascoded common source

.include /foss/pdks/sky130A/libs.tech/ngspice/corners/tt.spice
.param mc_mm_switch=0
.param lx=0.15 wx=162.5 nfx=40 idx=0.5m
.save @m.M1.msky130_fd_pr__nfet_01v8_lvt
.save @m.M2.msky130_fd_pr__nfet_01v8_lvt

* D G S B
M2 out vbias d 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
M1 d vin 0 0 sky130_fd_pr__nfet_01v8_lvt L={lx} W={wx} nf={nfx} ad='int((nf+1)/2) * W/nf * 0.29' as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)'
+ ps='2*int((nf+2)/2) * (W/nf + 0.29)' nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1

vg  vin  0  0.3
vb  vbias  0  1
is  out  0  {idx}

.control
  op
  *show
  print @m.M1.msky130_fd_pr__nfet_01v8_lvt[gm]
  print @m.M1.msky130_fd_pr__nfet_01v8_lvt[id]
  print @m.M1.msky130_fd_pr__nfet_01v8_lvt[gds]
.endc
.end

Writing ./sizing_cascoded_common_source.spice
